# SLU06 - Dealing with Data Problems

This notebook has exercises covering the following topics:

- Tidy Data
- Data Entry Problems
- Missing Values

In [44]:
import os
import pandas as pd
import numpy as np
import hashlib
import json

The dataset that we'll use for these exercises comes from the World Health Organisation, and records the counts of confirmed tuberculosis cases by country, year (between 1980 and 2008), and demographic group.
The demographic groups are broken down by sex (m, f) and age (0–14, 15–25, 25–34, 35–44, 45–54, 55–64, 65+, unknown).

But as you will see, this dataset doesn't follow the Tidy Data Principle.

In the following exercises, we will clean it.

First let's read the dataset into a pandas dataframe.

In [45]:
df = pd.read_csv(os.path.join('data', 'tb.csv'))
df.head(10)

,country,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,new_sp_m65,new_sp_mu,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AD,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AD,1996,0.0,0.0,0.0,4.0,1.0,0.0,0.0,NaN,0.0,1.0,1.0,0.0,0.0,1.0,0.0,NaN
7,AD,1997,0.0,0.0,1.0,2.0,2.0,1.0,6.0,NaN,0.0,1.0,2.0,3.0,0.0,0.0,1.0,NaN
8,AD,1998,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AD,1999,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN


### Exercise 1

Create a list with the column names that refer to the demographic groups data (i.e, those that start with `new_sp_`).

Call the list `demo_group_data`. The values in list should be in the same order as they are in the dataframe's columns.

In [46]:
demo_group_data = list(df[[i for i in df.columns if "new_sp_" in i]])



In [47]:
# Create a list with the demographic groups data
# demo_group_data = ...

# YOUR CODE HERE


demo_group_data = list(df[[col for col in df.columns if "new_sp_" in col]])




In [48]:
first_element_hash = 'c8ae9368386d7c77204db0840e77d432aff39941d6f42407b0164028b0bdd5c6'
entire_list_hash = '11db8cc5591bcd9757e052b44e7b1a5ccd4f2de2e5c3130d5437834ea021aa78'

assert isinstance(demo_group_data, list), "demo_group_data should be a list."

assert len(demo_group_data) == 16, "demo_group_data doesn't have the right number of elements."

error_msg = 'The first element of the list is not correct.'
assert first_element_hash == hashlib.sha256(bytes(demo_group_data[0], encoding='utf8')).hexdigest(), error_msg

error_msg = 'The list is not correct.'
assert entire_list_hash == hashlib.sha256(json.dumps(demo_group_data).encode()).hexdigest()

### Exercise 2

Create a new dataframe, that is the result of changing `df` so that it has 4 columns:
- country
- year
- demo_group
- cases

The values in the demo_group column should be exactly the ones that we currently have as column names.

The new dataframe should be called `df_tidy_1`.

Hint: use the answer from **Exercise 1**.

In [49]:
# Create a new version of df that doesn't have variable values as columns
# df_tidy_1 = ...

# YOUR CODE HERE

df_tidy_1 = pd.melt(df, id_vars= ['country','year'],  value_vars =  demo_group_data, var_name = 'demo_group', value_name= 'cases')

In [50]:
column_names_hash = '8a5277b30342779427d2bf91bdbe9669975eecaf85c3560103367a75af3d971c'
dataframe_shape_hash = 'd28d50e42c0b9b03762856921319b41c42f7fa4d1b5da5c5b4b84d716eff3a9c'
dataframe_hash = '1c564c7ad9e350444637e70de9af00877e99dc5518dc130664617e4ce51b1a42'

assert isinstance(df_tidy_1, pd.DataFrame), "df_tidy_1 should be a pandas DataFrame."

error_msg = "The dataframe doesn't have the right columns."
assert column_names_hash == hashlib.sha256(json.dumps(df_tidy_1.columns.tolist()).encode()).hexdigest(), error_msg

error_msg = "The dataframe doesn't have the right shape."
assert dataframe_shape_hash == hashlib.sha256(json.dumps(df_tidy_1.shape).encode()).hexdigest(), error_msg

error_msg = "The dataframe is not correct."
assert dataframe_hash == hashlib.sha256(df_tidy_1.to_msgpack()).hexdigest(), error_msg

### Exercise 3

Now, take a look at the values in column `demo_group`.

They all start with a meaningless "new_sp_" string, so we will remove it.

Create a new dataframe `df_tidy_2`, that is a copy of `df_tidy_1`, but where the `demo_group` column no longer has the "new_sp_" in each value. 

In [51]:
# Create a new version of df_tidy_1 that doesn't have "new_sp_" in the demo_group column
# df_tidy_2 = ...

# YOUR CODE HERE
df_tidy_1['demo_group'] = df_tidy_1['demo_group'].str.replace('new_sp_', '')
df_tidy_2 = pd.DataFrame(df_tidy_1)

In [52]:
column_names_hash = '8a5277b30342779427d2bf91bdbe9669975eecaf85c3560103367a75af3d971c'
dataframe_shape_hash = 'd28d50e42c0b9b03762856921319b41c42f7fa4d1b5da5c5b4b84d716eff3a9c'
dataframe_hash = '5d7508a0b5cda0e5f4cb530f5922199576d255ac7340d9fc1c89b28b5d6b355d'

assert isinstance(df_tidy_2, pd.DataFrame), "df_tidy_2 should be a pandas DataFrame."

error_msg = "The dataframe doesn't have the right columns."
assert column_names_hash == hashlib.sha256(json.dumps(df_tidy_2.columns.tolist()).encode()).hexdigest(), error_msg

error_msg = "The dataframe doesn't have the right shape."
assert dataframe_shape_hash == hashlib.sha256(json.dumps(df_tidy_2.shape).encode()).hexdigest(), error_msg

error_msg = "The dataframe is not correct."
assert dataframe_hash == hashlib.sha256(df_tidy_2.to_msgpack()).hexdigest(), error_msg

### Exercise 4

As you may have noticed, our dataset still has a problem. The `demo_group` column has data that in fact represents two variables: `gender` and `age`.

So our end goal will be to replace the `demo_group` column with the two new columns.

On this exercise, create a new dataframe `df_tidy_3`, that is a copy of `df_tidy_2`, but has a new column `gender`, which has the first letter of column `demo_group` (m/f).

Hint: you may need to search for this one online, as you'll need a string method that we didn't cover in the Learning notebook.

In [53]:
# Create a new version of df_tidy_2 that has a new column gender
# df_tidy_3 = ...

# YOUR CODE HERE

df_tidy_2['gender'] = df_tidy_2['demo_group'].astype(str).str[0]
df_tidy_3 = pd.DataFrame(df_tidy_2)

In [54]:
column_names_hash = 'a3efb4e4eb3a43fa947a95032395ccc21d2cc3715a8d359e4c711132f4b837e0'
dataframe_shape_hash = '78cdfddff871199ede18988405d0daf7c840512ff0bdb01168d3f85394618ef8'
dataframe_hash = '282c81e0a303c5515386d4d78bbd2307377447e9f14ea35d6bcbdc2ec720b1bb'

assert isinstance(df_tidy_3, pd.DataFrame), "df_tidy_3 should be a pandas DataFrame."

error_msg = "The dataframe doesn't have the right columns."
assert column_names_hash == hashlib.sha256(json.dumps(df_tidy_3.columns.tolist()).encode()).hexdigest(), error_msg

error_msg = "The dataframe doesn't have the right shape."
assert dataframe_shape_hash == hashlib.sha256(json.dumps(df_tidy_3.shape).encode()).hexdigest(), error_msg

error_msg = "The dataframe is not correct."
assert dataframe_hash == hashlib.sha256(df_tidy_3.to_msgpack()).hexdigest(), error_msg

### Exercise 5

Now we want to create the column `age` and get rid of column `demo_group`.

On this exercise, create a new dataframe `df_tidy_4`, that is a copy of `df_tidy_3`, but has a new column `age`, which is the same as column `demo_group`, except that it has the first letter removed.

We also want to get rid of column `demo_group`, so make sure the new dataframe doesn't have it!

Hint: you may need to search for this one online, as you'll need a string method that we didn't cover in the Learning notebook.

In [55]:
# Create a new version of df_tidy_3 that has a new column age, and doesn't have column demo_group
# df_tidy_4 = ...

# YOUR CODE HERE
df_tidy_3['age'] = df_tidy_2['demo_group'].astype(str).str[1:]
df_tidy_3 = df_tidy_3.drop(['demo_group'], axis = 1)
df_tidy_4 = pd.DataFrame(df_tidy_3)


In [56]:
column_names_hash = 'ab05d50deb2a842b806a607ad26166c82f91960bd9dae704d8a11ff78fb60860'
dataframe_shape_hash = '78cdfddff871199ede18988405d0daf7c840512ff0bdb01168d3f85394618ef8'
dataframe_hash = 'f534bc81fdd39b7a79c0630d6a3a0728b89c40d6c95bf297d94e537bfa305f84'

assert isinstance(df_tidy_4, pd.DataFrame), "df_tidy_4 should be a pandas DataFrame."

error_msg = "The dataframe doesn't have the right columns."
assert column_names_hash == hashlib.sha256(json.dumps(df_tidy_4.columns.tolist()).encode()).hexdigest(), error_msg

error_msg = "The dataframe doesn't have the right shape."
assert dataframe_shape_hash == hashlib.sha256(json.dumps(df_tidy_4.shape).encode()).hexdigest(), error_msg

error_msg = "The dataframe is not correct."
assert dataframe_hash == hashlib.sha256(df_tidy_4.to_msgpack()).hexdigest(), error_msg

### Exercise 6

If you take a look at the age values, they are quite hard to understand... 
Let's make those easier to grasp!

Create a new dataframe `df_tidy_5`, where the `age` column has more understandable values.

We want that each age interval is separated by a `-`. For instance, `014` should be replaced with `0-14`.
We also want the last age group to be represented as `65+`. The unknown values `u` can be left unchanged.

In [57]:
# Create a new version of df_tidy_4 where the age values are more understandable
# df_tidy_5 = ...

# YOUR CODE HERE
ca = []
for i in df_tidy_4['age']:
    if i == 'u':
        ca.append(i)
    elif i == '65':
        ca.append(i + '+')
    else:
        ca.append(str(i[:len(i)-2]) + '-' + i[-2:])

df_tidy_4['age'] = ca
df_tidy_5 = pd.DataFrame(df_tidy_4)      
        
print(df_tidy_5)


      country  year  cases gender   age
0          AD  1989    NaN      m  0-14
1          AD  1990    NaN      m  0-14
2          AD  1991    NaN      m  0-14
3          AD  1992    NaN      m  0-14
4          AD  1993    NaN      m  0-14
5          AD  1994    NaN      m  0-14
6          AD  1996    0.0      m  0-14
7          AD  1997    0.0      m  0-14
8          AD  1998    0.0      m  0-14
9          AD  1999    0.0      m  0-14
10         AD  2000    0.0      m  0-14
11         AD  2001    0.0      m  0-14
12         AD  2002    0.0      m  0-14
13         AD  2003    0.0      m  0-14
14         AD  2004    0.0      m  0-14
15         AD  2005    0.0      m  0-14
16         AD  2006    0.0      m  0-14
17         AD  2007    NaN      m  0-14
18         AD  2008    0.0      m  0-14
19         AE  1980    NaN      m  0-14
20         AE  1981    NaN      m  0-14
21         AE  1982    NaN      m  0-14
22         AE  1983    NaN      m  0-14
23         AE  1984    NaN      m  0-14


In [58]:
column_names_hash = 'ab05d50deb2a842b806a607ad26166c82f91960bd9dae704d8a11ff78fb60860'
dataframe_shape_hash = '78cdfddff871199ede18988405d0daf7c840512ff0bdb01168d3f85394618ef8'
dataframe_hash = '324f52dadcd96598cd0edd64ce2ea6717aa7263a105a4b53b7ee5f6ea8baee30'

assert isinstance(df_tidy_4, pd.DataFrame), "df_tidy_5 should be a pandas DataFrame."

error_msg = "The dataframe doesn't have the right columns."
assert column_names_hash == hashlib.sha256(json.dumps(df_tidy_5.columns.tolist()).encode()).hexdigest(), error_msg

error_msg = "The dataframe doesn't have the right shape."
assert dataframe_shape_hash == hashlib.sha256(json.dumps(df_tidy_5.shape).encode()).hexdigest(), error_msg

error_msg = "The dataframe is not correct."
assert dataframe_hash == hashlib.sha256(df_tidy_5.to_msgpack()).hexdigest(), error_msg

### Exercise 7

Now that our dataset follows the Tidy Data principles, let's check for duplicates.

Assign to variable `duplicates_count` the number of duplicates in `df_tidy_5`. Make sure the value you get is an integer.

In [59]:
# Count the number of duplicates in df_tidy_5
# duplicates_count = ...
# YOUR CODE HERE
duplicates = df_tidy_5.duplicated(keep='first')
duplicates_count = int(duplicates.sum())



In [60]:
duplicates_count_hash = 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'

error_msg = "duplicates_count must be an integer (explicitly convert it to 'int' if it is type 'numpy.int64')"
assert isinstance(duplicates_count, int), error_msg

error_msg = "duplicates_count doesn't have the right value."
assert duplicates_count_hash == hashlib.sha256(bytes(duplicates_count)).hexdigest(), error_msg

### Exercise 8

Now let's count the number of rows with missing values in our `df_tidy_5` dataframe.

Assign that value to variable `null_count` and make sure it is an integer.

Also take a moment to think about the number of null values we had in this dataset, in comparison to the dataset size.

In [61]:
# Count the number of null values in df_tidy_5
# null_count = ...

# YOUR CODE HERE
n_count_row = df_tidy_5.isnull().any(axis=1)
null_count = int(n_count_row.values.sum())

In [62]:
duplicates_count_hash = 'e41482b02884d9b42b5ea9abb63be3658c28266e8f62f31db33fc9efa53cf01d'

error_msg = "null_count must be an integer (explicitly convert it to 'int' if it is type 'numpy.int64')"
assert isinstance(null_count, int), error_msg

error_msg = "null_count doesn't have the right value."
assert duplicates_count_hash == hashlib.sha256(bytes(null_count)).hexdigest(), error_msg

### Exercise 9

Now let's use a common technique to imput missing values: replacing them with the mean.

Create a new dataframe `df_tidy_6`, which is a copy of `df_tidy_5`, except that the missing values in the `cases` column are replaced with the mean of the column.

Also, take a moment to think if this is a good strategy in this case.

In [63]:
df_tidy_5['cases'] = df_tidy_5.cases.fillna(df_tidy_5.cases.mean())
df_tidy_6 = pd.DataFrame(df_tidy_5)
print(df_tidy_6)

      country  year       cases gender   age
0          AD  1989  636.764374      m  0-14
1          AD  1990  636.764374      m  0-14
2          AD  1991  636.764374      m  0-14
3          AD  1992  636.764374      m  0-14
4          AD  1993  636.764374      m  0-14
5          AD  1994  636.764374      m  0-14
6          AD  1996    0.000000      m  0-14
7          AD  1997    0.000000      m  0-14
8          AD  1998    0.000000      m  0-14
9          AD  1999    0.000000      m  0-14
10         AD  2000    0.000000      m  0-14
11         AD  2001    0.000000      m  0-14
12         AD  2002    0.000000      m  0-14
13         AD  2003    0.000000      m  0-14
14         AD  2004    0.000000      m  0-14
15         AD  2005    0.000000      m  0-14
16         AD  2006    0.000000      m  0-14
17         AD  2007  636.764374      m  0-14
18         AD  2008    0.000000      m  0-14
19         AE  1980  636.764374      m  0-14
20         AE  1981  636.764374      m  0-14
21        

In [64]:
# Create a new version of df_tidy_5 where the cases missing values are replaced with the mean
# df_tidy_6 = ...

# YOUR CODE HERE

df_tidy_5['cases'] = df_tidy_5.cases.fillna(df_tidy_5.cases.mean())
df_tidy_6 = pd.DataFrame(df_tidy_5)


In [65]:
column_names_hash = 'ab05d50deb2a842b806a607ad26166c82f91960bd9dae704d8a11ff78fb60860'
dataframe_shape_hash = '78cdfddff871199ede18988405d0daf7c840512ff0bdb01168d3f85394618ef8'
dataframe_hash = '1f8e04b67a127979a1a47c9aac44716d9c08b9aa0d7641b7bb8542769a32db37'

assert isinstance(df_tidy_6, pd.DataFrame), "df_tidy_6 should be a pandas DataFrame."

error_msg = "The dataframe doesn't have the right columns."
assert column_names_hash == hashlib.sha256(json.dumps(df_tidy_6.columns.tolist()).encode()).hexdigest(), error_msg

error_msg = "The dataframe doesn't have the right shape."
assert dataframe_shape_hash == hashlib.sha256(json.dumps(df_tidy_6.shape).encode()).hexdigest(), error_msg

error_msg = "The dataframe is not correct."
assert dataframe_hash == hashlib.sha256(df_tidy_6.to_msgpack()).hexdigest(), error_msg

### Exercise 10

Did you think that replacing the missing values with the mean was a terrible idea in this case? Because it is!

Let's see why.

Create a new dataframe `top_cases`, by:
- sorting `df_tidy_5` by the cases column in descending order
- taking the first row per country, using the drop_duplicates function
- taking the first 10 rows of that dataframe

Then take a look at the countries in `top_countries`. Can you see a correlation between the number of cases and the population size of those countries?

In [66]:
# Create a dataframe with the rows that correspond to the 10 highest number of cases in df_tidy_5
# top_cases = ...

# YOUR CODE HERE


top_cases = pd.DataFrame()
top_cases = top_cases.head(10)

In [67]:
column_names_hash = 'ab05d50deb2a842b806a607ad26166c82f91960bd9dae704d8a11ff78fb60860'
dataframe_shape_hash = '36d0cc685f3df22cfee85b856d878e734000d5cf3ff3cc3f809a96c623410ff4'
dataframe_hash = '8420df6d826597ab69f60b0c1131a440908a1fbe7ed5abdb673c17040ad72f4a'

assert isinstance(top_cases, pd.DataFrame), "top_cases should be a pandas DataFrame."

error_msg = "The dataframe doesn't have the right columns."
assert column_names_hash == hashlib.sha256(json.dumps(top_cases.columns.tolist()).encode()).hexdigest(), error_msg

error_msg = "The dataframe doesn't have the right shape."
assert dataframe_shape_hash == hashlib.sha256(json.dumps(top_cases.shape).encode()).hexdigest(), error_msg

error_msg = "The dataframe is not correct."
assert dataframe_hash == hashlib.sha256(top_cases.to_msgpack()).hexdigest(), error_msg

AssertionError: The dataframe doesn't have the right columns.

### Exercise 11 - ungraded

Take some time to discuss with a coleague sitting next to you, what would be a better way to impute missing values in this dataset.